<a href="https://colab.research.google.com/github/shivam-39/abbreviation-identification/blob/main/src/categorization_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
import re
import random

In [ ]:
import os

In [ ]:
!pip install -q tensorflow_datasets
!pip install -q tensorflow_text

     |████████████████████████████████| 4.9 MB 12.8 MB/s 


In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/Colab Notebooks/full_data.csv'

In [ ]:
data = pd.read_csv(path,nrows=1000)

In [ ]:
# data2 = pd.read_csv(path, nrows=3000000)

In [ ]:
# len(data2)

In [ ]:
len(data)

1000

In [ ]:
data.head()

,TEXT,LOCATION,LABEL
0,alphabisabolol has a primary antipeptic action...,56,substrate
1,a report is given on the recent discovery of o...,24|49|68|113|137|172,carcinosarcoma|recovery|reference|recovery|aft...
2,the virostatic compound nndiethyloxotetradecyl...,55,substrate
3,rmi rmi and rmi are newly synthetized nrdibenz...,25|82|127|182|222,compounds|compounds|inhibitory|lethal doses|ca...
4,a doubleblind study with intraindividual compa...,22|26|28|77|90|144|158|203,oxazepam|placebo|oral administration|pentagast...


In [ ]:
data['TEXT'][3]

'rmi rmi and rmi are newly synthetized nrdibenzobfoxepinylnmethylpiperazinemaleates which show interesting psychopharmacologic effects this work contains the results of a study performed with these three EDC in order to demonstrate their neuropsycholeptic activity in comparison with chloropromazine cpz and chlordiazepoxide cpd the inhibition of motility observed in mice shows that the compounds reduce the normal spontaneous motility as well as the muscle tone the centraldepressant activity is evidenced by increased barbiturateinduced sleep and a remarkable eyelid ptosis can also be observed our EDC do not show any activity on electroshock just as do cpz and cpd as to the antipsychotic outline our compounds show strong reduction of lethality due to amphetamine in grouped mice and a strong antiapomorphine activity they show also an antiaggressive effect and an GABA activity on avoidance behaviour much stronger than cpz we have also found extrapyramidal effects as catalepsy common to many

In [ ]:
data['LABEL'][3]

'compounds|compounds|inhibitory|lethal doses|catecholamines'

In [ ]:
words = data['TEXT'][3].split()
abrr_label = data['LABEL'][3].split('|')
nums = data['LOCATION'][3].split('|')
nums = [int(i) for i in nums]
for i in range(len(nums)):
    print(words[nums[i]]+" "+abrr_label[i])

EDC compounds
EDC compounds
GABA inhibitory
MLD lethal doses
CAs catecholamines


In [ ]:
final_data = data.sample(frac = 1)
len(final_data)

1000000

In [ ]:
final_data = final_data[:65000]
len(final_data)

65000

In [ ]:
data_train = final_data[:60000]
data_test = final_data[60000:65000]

In [ ]:
print(data_train.shape)
print(data_test.shape)
print(final_data.shape)

(60000, 3)
(5000, 3)
(65000, 3)


In [ ]:
dic = {}
for i in range(len(final_data)):
    words = data['LABEL'][i].split('|')
    for word in words:
        if word in dic:
            dic[word]+=1
        else:
            dic[word]=1

In [ ]:
import operator
sorted_d = sorted(dic.items(), key=operator.itemgetter(1),reverse=True)
sorted_d[0]

('after', 8255)

In [ ]:
top_20 = []
for i in range(20):
    top_20.append(sorted_d[i][0])

In [ ]:
# top_20

In [ ]:
data_train.reset_index(drop = True, inplace = True)
data_test.reset_index(drop = True, inplace = True)

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# stop_words

In [ ]:
data_test.head(10)

,TEXT,LOCATION,LABEL
0,we conducted a doubleblind comparative study o...,83|93|106,after|terbinafine|griseofulvin
1,a CS of plasmodium vivax from thailand with a ...,1|49|65|67,strain|intact|infected|feeding
2,differentiated hl granulocytes were used to st...,65|71|183,release|membranes|peptides
3,a cultivation system has been developed for pe...,80|92|120|161|196|207,after|branch|plasma membrane|plasma membrane|d...
4,chronic nonspecific diarrhea cnsd is the most ...,63|119,groups|symptoms
5,the data to be discussed in this paper are dra...,28|54|114|153,preterm|study|preterm|development
6,three alternative methods for the screening of...,14|142|153|157|200,preparations|evaluation|assay|assay|behavior
7,nuclear sections of basalioma cells nuclear se...,54,tumor
8,this article intends to present goals tasks an...,54,development
9,almost all patients with cataplectic narcoleps...,58,study


In [ ]:
sentences_train = []
abber_locs_train = []
abber_category_train = []

sentences_test = []
abber_locs_test = []
abber_category_test = []

for i in range(0, len(data_train)):
    sentences_train.append(data_train['TEXT'][i])
    abber_locs_train.append(data_train['LOCATION'][i].split("|"))
    abber_category_train.append(data_train['LABEL'][i].split("|"))
    
for i in range(0, len(data_test)):
    sentences_test.append(data_test['TEXT'][i])
    abber_locs_test.append(data_test['LOCATION'][i].split("|"))
    abber_category_test.append(data_test['LABEL'][i].split("|"))

In [ ]:
sentences_test[9]

'almost all patients with cataplectic narcolepsy are drpositive it has been suggested that the nondr allelehaplotype might not be neutral with respect to disease susceptibility it has also been reported that taq i dqa and bam hi eco ri eco rv and pst i dqb restriction fragments might differentiate between narcoleptic and healthy drpositive individuals in the present T0 hla class ii gene polymorphisms were investigated by restriction fragment length polymorphism rflp analysis in swedish patients with cataplectic narcolepsy random controls and drassociated homozygous cell lines all patients had taq i drbdqadqb patterns corresponding to the drwdqwdw haplotype the nondr haplotype was found to be neutral this genotyped group of patients allows firm rejection of a recessive mode of inheritance and supports a dominant or additive model no dqa or dqb rflps were found that could differentiate between drpositive narcoleptics drwdqwdwpositive controls or dwhomozygous cell lines no significant msp

In [ ]:
abber_locs_test[9]

['58']

In [ ]:
abber_category_test[9]

['study']

In [ ]:
print(len(sentences_test))
print(len(abber_locs_test))
print(len(abber_category_test))

5000
5000
5000


In [ ]:
for i in range(len(abber_category_train)):
    for j in range(len(abber_category_train[i])):
        if abber_category_train[i][j] not in top_20:
            abber_category_train[i][j]='-'

for i in range(len(abber_category_test)):
    for j in range(len(abber_category_test[i])):
        if abber_category_test[i][j] not in top_20:
            abber_category_test[i][j]='-'

In [ ]:
len(abber_locs_test[9])

1

In [ ]:
x0 = []
x1 = []
x2 = []
for i in range(len(abber_category_train)):
    y0 = []
    y1 = []
    y2 = []
    for j in range(len(abber_category_train[i])):
        if abber_category_train[i][j]!='-':
            y1.append(abber_locs_train[i][j])
            y2.append(abber_category_train[i][j])
    if len(y1)!=0:
        x0.append(sentences_train[i])
        x1.append(y1)
        x2.append(y2)

sentences_train = x0
abber_locs_train = x1
abber_category_train = x2


x0 = []
x1 = []
x2 = []
for i in range(len(abber_category_test)):
    y0 = []
    y1 = []
    y2 = []
    for j in range(len(abber_category_test[i])):
        if abber_category_test[i][j]!='-':
            y1.append(abber_locs_test[i][j])
            y2.append(abber_category_test[i][j])
    if len(y1)!=0:
        x0.append(sentences_test[i])
        x1.append(y1)
        x2.append(y2)

sentences_test = x0
abber_locs_test = x1
abber_category_test = x2

In [ ]:
Y_vec = []
for label_vec in abber_category_train:
    temp = []
    for label in label_vec:
        temp2 = [0]*20
        for i in range(len(top_20)):
            if label==top_20[i]:
                temp2[i]=1
        temp.append(temp2)
    Y_vec.append(temp)

Y_vec_test = []
for label_vec in abber_category_test:
    temp = []
    for label in label_vec:
        temp2 = [0]*20
        for i in range(len(top_20)):
            if label==top_20[i]:
                temp2[i]=1
        temp.append(temp2)
    Y_vec_test.append(temp)

In [ ]:
Y_vec_test[0]

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# sentences_train
# abber_locs_train
# abber_category_train

In [ ]:
abber_category_train[0]

['active']

In [ ]:
avg = 0
for sent in sentences_train:
    disc = sent.split()
    x = len(disc)
    avg += x
avg=avg/len(sentences_train)

print(avg)

167.8995605102369


In [ ]:
fd_sentences = []
fd_labels = []
fd_words = []
for i in range(len(sentences_train)):
    words_vec = sentences_train[i].split()
    sent_len = len(words_vec)
    for j in range(len(abber_locs_train[i])):
        pos = int(abber_locs_train[i][j])
        if sent_len<128:
            fd_sentences.append(" ".join(words_vec))
        elif pos<64:
            fd_sentences.append(" ".join(words_vec[:128]))
        else:
            fd_sentences.append(" ".join(words_vec[pos-64:pos+63]))
        fd_labels.append(abber_category_train[i][j])
        fd_words.append(words_vec[pos])

fd_sentences_test = []
fd_labels_test = []
fd_words_test = []
for i in range(len(sentences_test)):
    words_vec = sentences_test[i].split()
    sent_len = len(words_vec)
    for j in range(len(abber_locs_test[i])):
        pos = int(abber_locs_test[i][j])
        if sent_len<128:
            fd_sentences_test.append(" ".join(words_vec))
        elif pos<64:
            fd_sentences_test.append(" ".join(words_vec[:128]))
        else:
            fd_sentences_test.append(" ".join(words_vec[pos-64:pos+63]))
        fd_labels_test.append(abber_category_test[i][j])
        fd_words_test.append(words_vec[pos])

In [ ]:
print(len(fd_sentences_test))
print(len(fd_labels_test))
print(len(fd_words_test))

3634
3634
3634


In [ ]:
# print(fd_sentences.shape)
# fd_labels

In [ ]:
for i in range(5):
    print(fd_sentences[i])
    print(fd_labels[i])

days of age a control group cont remained with the mother in the nest during this time cold subjects were developmentally delayed and had lower body and brain weights than the other three groups into adulthood warm and agit subjects both maternally separated at nest temperature had significant growth delays compared to cont but grew more quickly than cold subjects cold subjects were less AS than the other maternally separated groups and warm and agit groups were more active activity did not differ at or days of age however AD warm subjects were less sensitive and cold subjects were more sensitive to amphetamine as measured by locomotor activity than cont and agit subjects who did not differ from each other the relationship between early stress changes in
active
region an open reading frame encoding the expected amino acid protein and an extensive nontranslated CS the ID of the clone as full length was consistent with results obtained in mrna blotting experiments using polya mrna from b

In [ ]:
Y_vec = []
for label in fd_labels:
    temp = [0]*20
    for i in range(len(top_20)):
        if label==top_20[i]:
            temp[i]=1
    Y_vec.append(temp)

Y_vec_test = []
for label in fd_labels_test:
    temp = [0]*20
    for i in range(len(top_20)):
        if label==top_20[i]:
            temp[i]=1
    Y_vec_test.append(temp)

In [ ]:
len(Y_vec_test)

3634

In [ ]:
Y_vec2 = []
for label in fd_labels:
    for i in range(len(top_20)):
        if label==top_20[i]:
            temp=i
    Y_vec2.append(temp)

Y_vec2_test = []
for label in fd_labels_test:
    for i in range(len(top_20)):
        if label==top_20[i]:
            temp=i
    Y_vec2_test.append(temp)

In [ ]:
Y_vec2_test[2]

0

In [ ]:
len(Y_vec2_test)

3634

In [ ]:
abbers_train = []
for i in range(0, len(abber_locs_train)):
    temp_train = []
    for l in abber_locs_train[i]:
        temp_train.append(sentences_train[i].split()[int(l)])
    abbers_train.append(temp_train)
    
abbers_test = []
for i in range(0, len(abber_locs_test)):
    temp_test = []
    for l in abber_locs_test[i]:
        temp_test.append(sentences_test[i].split()[int(l)])
    abbers_test.append(temp_test)

In [ ]:
abbers_train[0]

['AS']

In [ ]:
def select_words_train(sentence, abber_list):
  j = 0
  words = sentence.split()
  random.shuffle(words)
  temp = []
  for i in range(0, len(words)):
    if (words[i] in abber_list or words[i] in temp):
      continue
    else:
      temp.append(words[i])
      j += 1
    if (j == (9*len(abber_list))):
      break
  return temp


def select_words_test(sentence, abber_list):
  j = 0
  words = sentence.split()
  random.shuffle(words)
  temp = []
  for i in range(0, len(words)):
    if (words[i] in abber_list or words[i] in temp):
      continue
    else:
      temp.append(words[i])
      j += 1
  return temp

In [ ]:
normal_words_train = []
for i in range(0, len(sentences_train)):
  normal_words_train.append(select_words_train(sentences_train[i], abbers_train[i]))

normal_words_test = []
for i in range(0, len(sentences_test)):
  normal_words_test.append(select_words_test(sentences_test[i], abbers_test[i]))

In [ ]:
sentences_train[1]

'the first representative of a group of mammalian low molecular weight phosphotyrosyl protein phosphatases was cloned sequenced and expressed in escherichia coli using a mer oligonucleotide probe based on the amino acid sequence of the purified enzyme several overlapping cdna clones were isolated from a bovine HR cdna library a fulllength clone was obtained consisting of a bp noncoding region an open reading frame encoding the expected amino acid protein and an extensive nontranslated CS the ID of the clone as full length was consistent with results obtained in mrna blotting experiments using polya mrna from bovine heart the coding sequence was placed downstream of a bacteriophage t promoter and protein was expressed in e coli the expressed enzyme was soluble and catalytically AS and was readily isolated and purified the recombinant protein had the expected mr of estimated by sdspage and it showed crossreactivity with antisera that had been raised against both the bovine heart and the 

In [ ]:
def text_cleaner(text):
    new_string = str(text)
    return new_string

def abbreviation_cleaner(text):
    new_string = str(text)
    return new_string

In [ ]:
cleaned_sentences_train = []
cleaned_abbers_train = []
cleaned_words_train = []

cleaned_sentences_test = []
cleaned_abbers_test = []
cleaned_words_test = []

for i in range(0, len(sentences_train)):
  cleaned_sentences_train.append(text_cleaner(sentences_train[i]))

for i in range(0, len(abbers_train)):
  temp = []
  for abber in abbers_train[i]:
    temp.append(abbreviation_cleaner(abber))
  cleaned_abbers_train.append(temp)

for i in range(0, len(normal_words_train)):
  temp = []
  for word in normal_words_train[i]:
    temp.append(abbreviation_cleaner(word))
  cleaned_words_train.append(temp)



for i in range(0, len(sentences_test)):
  cleaned_sentences_test.append(text_cleaner(sentences_test[i]))

for i in range(0, len(abbers_test)):
  temp = []
  for abber in abbers_test[i]:
    temp.append(abbreviation_cleaner(abber))
  cleaned_abbers_test.append(temp)

for i in range(0, len(normal_words_test)):
  temp = []
  for word in normal_words_test[i]:
    temp.append(abbreviation_cleaner(word))
  cleaned_words_test.append(temp)

In [ ]:
abber_vicinity_train = []
for i in range(0, len(cleaned_sentences_train)):
  list_of_words = cleaned_sentences_train[i].split()
  vicinity_sentences = []
  for abber in cleaned_abbers_train[i]:
    # print(i)
    index = list_of_words.index(abber)
    lower_limit = max(0, index-4)
    upper_limit = min(index + 4 + 1, len(list_of_words))
    sentence = ' '.join(list_of_words[lower_limit:upper_limit]).strip()
    vicinity_sentences.append(sentence)
  abber_vicinity_train.append(vicinity_sentences)

abber_vicinity_test = []
for i in range(0, len(cleaned_sentences_test)):
  list_of_words = cleaned_sentences_test[i].split()
  vicinity_sentences = []
  for abber in cleaned_abbers_test[i]:
    # print(i)
    index = list_of_words.index(abber)
    lower_limit = max(0, index-4)
    upper_limit = min(index + 4 + 1, len(list_of_words))
    sentence = ' '.join(list_of_words[lower_limit:upper_limit]).strip()
    vicinity_sentences.append(sentence)
  abber_vicinity_test.append(vicinity_sentences)

In [ ]:
normal_words_vicinity_train = []
for i in range(0, len(cleaned_sentences_train)):
  list_of_words = cleaned_sentences_train[i].split()
  vicinity_sentences = []
  for word in cleaned_words_train[i]:
    # print(i)
    index = list_of_words.index(word)
    lower_limit = max(0, index-4)
    upper_limit = min(index + 4 + 1, len(list_of_words))
    sentence = ' '.join(list_of_words[lower_limit:upper_limit]).strip()
    vicinity_sentences.append(sentence)
  normal_words_vicinity_train.append(vicinity_sentences)

normal_words_vicinity_test = []
for i in range(0, len(cleaned_sentences_test)):
  list_of_words = cleaned_sentences_test[i].split()
  vicinity_sentences = []
  for word in cleaned_words_test[i]:
    # print(i)
    index = list_of_words.index(word)
    lower_limit = max(0, index-4)
    upper_limit = min(index + 4 + 1, len(list_of_words))
    sentence = ' '.join(list_of_words[lower_limit:upper_limit]).strip()
    vicinity_sentences.append(sentence)
  normal_words_vicinity_test.append(vicinity_sentences)

In [ ]:
abber_vicinity_train[1]

['was soluble and catalytically AS and was readily isolated']

In [ ]:
final_normal_sentences_train = []
final_normal_words_train = []
for i in range(0, len(normal_words_vicinity_train)):
  for j in range(0, len(normal_words_vicinity_train[i])):
    final_normal_sentences_train.append(normal_words_vicinity_train[i][j])
    final_normal_words_train.append(cleaned_words_train[i][j])
    
final_normal_sentences_test = []
final_normal_words_test = []
for i in range(0, len(normal_words_vicinity_test)):
  for j in range(0, len(normal_words_vicinity_test[i])):
    final_normal_sentences_test.append(normal_words_vicinity_test[i][j])
    final_normal_words_test.append(cleaned_words_test[i][j])

In [ ]:
final_abber_sentences_train = []
final_abber_words_train = []
for i in range(0, len(abber_vicinity_train)):
  for j in range(0, len(abber_vicinity_train[i])):
    final_abber_sentences_train.append(abber_vicinity_train[i][j])
    final_abber_words_train.append(cleaned_abbers_train[i][j])
    
final_abber_sentences_test = []
final_abber_words_test = []
for i in range(0, len(abber_vicinity_test)):
  for j in range(0, len(abber_vicinity_test[i])):
    final_abber_sentences_test.append(abber_vicinity_test[i][j])
    final_abber_words_test.append(cleaned_abbers_test[i][j])

In [ ]:
labels_train = []
for i in range(0, len(final_normal_words_train)):
  labels_train.append(0)
for i in range(0, len(final_abber_words_train)):
  labels_train.append(1)

labels_test = []
for i in range(0, len(final_normal_words_test)):
  labels_test.append(0)
for i in range(0, len(final_abber_words_test)):
  labels_test.append(1)

In [ ]:
final_sentences_train = np.concatenate((np.array(final_normal_sentences_train), np.array(final_abber_sentences_train)), axis = 0)
final_sentences_test = np.concatenate((np.array(final_normal_sentences_test), np.array(final_abber_sentences_test)), axis = 0)

In [ ]:
final_words_train = np.concatenate((np.array(final_normal_words_train), np.array(final_abber_words_train)), axis = 0)
final_words_test = np.concatenate((np.array(final_normal_words_test), np.array(final_abber_words_test)), axis = 0)

In [ ]:
final_labels_train = np.array(labels_train)
final_labels_test = np.array(labels_test)

In [ ]:
final_sentences_train[2]

'during this time cold subjects were developmentally delayed and'

In [ ]:
final_data_train = pd.DataFrame({'sentences': fd_sentences, 'words': fd_words, 'labels': Y_vec2})
final_data_test = pd.DataFrame({'sentences': fd_sentences_test, 'words': fd_words_test, 'labels': Y_vec2_test})

In [ ]:
final_data_train.head()

,sentences,words,labels
0,days of age a control group cont remained with...,AS,8
1,region an open reading frame encoding the expe...,AS,8
2,twelve outbreaks of SH b virus hbv infection a...,L1,17
3,a gallbladder cannula suitable for chronic imp...,CF,11
4,a gallbladder cannula suitable for chronic imp...,T3,0


In [ ]:
final_data_test.head()

,sentences,words,labels
0,we conducted a doubleblind comparative study o...,T3,0
1,hl granulocytes were used to study the mechani...,PR,10
2,spherical growth of spores was initiated by in...,T3,0
3,segregation of the conidium from the phialide ...,SC,18
4,chronic nonspecific diarrhea cnsd is the most ...,CG,7


In [ ]:
# for i in range(len(labels_test)):
#     if label_test[i]==1:
#         print(final_words_test[i])

In [ ]:
final_data_train['sentences'][4]

'a gallbladder cannula suitable for chronic implantation in the dog is described the device is simple to operate inexpensive to construct durable and easy to implant with the cannula in place measurements of hepatic and biliary drug kinetics as well as enterohepatic CF in the awake nonstressed animal are possible minimal postoperative care is required T3 implantation of the device'

In [ ]:
# type(final_data_train_labels[0])

In [ ]:
# final_data_train['labels'].value_counts()

In [ ]:
final_data_test['labels'].value_counts()

0     716
3     384
2     345
4     202
7     197
1     187
5     174
11    146
6     143
9     136
19    125
16    122
15    119
17    117
14    110
13    101
8     100
12     80
10     79
18     51
Name: labels, dtype: int64

# Training the model on MeDal(train set) and testing on MeDal(test set)

In [ ]:
final_data_train['sentences'] = final_data_train['sentences'].apply(lambda x: str(x))
final_data_train['words'] = final_data_train['words'].apply(lambda x: str(x))

In [ ]:
final_data_train_labels = list(final_data_train['labels'])

In [ ]:
final_data_train_labels = np.array(final_data_train_labels)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

preprocess_lnk = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
enc_lnk = 'https://tfhub.dev/google/experts/bert/pubmed/2'
enc_lnk2 = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'

bert_preprocessor = hub.load(preprocess_lnk)

def make_classifier_model():
  text_input1 = layers.Input(shape = (), dtype = tf.string, name = 'text1')
  tokenizer_bert1 = hub.KerasLayer(bert_preprocessor.tokenize, name = 'tokenizer1')
  segments1 = tokenizer_bert1(text_input1)
#   packer1 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs, arguments = dict(seq_length = 30))
  packer1 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs)
  encoder_inputs1 = packer1([segments1])
  bert_encoder1 = hub.KerasLayer(enc_lnk2, trainable = False, name = 'BERT_encoder1')
  out1 = bert_encoder1(encoder_inputs1)
  net1 = out1['sequence_output']
  model1 = tf.keras.Sequential()
  model1.add(layers.LSTM(128,return_sequences=True))
  model1.add(layers.LSTM(64,return_sequences=True))
  model1.add(layers.LSTM(64))
  net1 = model1(net1)

#   text_input2 = layers.Input(shape = (), dtype = tf.string, name = 'text2')
#   tokenizer_bert2 = hub.KerasLayer(bert_preprocessor.tokenize, name = 'tokenizer2')
#   segments2 = tokenizer_bert2(text_input2)
#   packer2 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs, arguments = dict(seq_length = 5))
#   encoder_inputs2 = packer2([segments2])
#   bert_encoder2 = hub.KerasLayer(enc_lnk2, trainable = False, name = 'BERT_encoder2')
#   out2 = bert_encoder2(encoder_inputs2)
#   net2 = out2['sequence_output']
#   model2 = tf.keras.Sequential()
#   model2.add(layers.LSTM(100,return_sequences=True))
#   model2.add(layers.LSTM(40,return_sequences=True))
#   model2.add(layers.LSTM(10))
#   net2 = model2(net2)

  text_input3 = layers.Input(shape = (), dtype = tf.string, name = 'text3')
  tokenizer_bert3 = hub.KerasLayer(bert_preprocessor.tokenize, name = 'tokenizer3')
  segments3 = tokenizer_bert3(text_input3)
  packer3 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs)
#   packer3 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs, arguments = dict(seq_length = 5))
  encoder_inputs3 = packer3([segments3])
  bert_encoder3 = hub.KerasLayer(enc_lnk, trainable = False, name = 'BERT_encoder3')
  out3 = bert_encoder3(encoder_inputs3)
  net3 = out3['sequence_output']
  model3 = tf.keras.Sequential()
  model3.add(layers.LSTM(128,return_sequences=True))
  model3.add(layers.LSTM(64,return_sequences=True))
  model3.add(layers.LSTM(64))
  net3 = model3(net3)

  final_input = layers.Concatenate(axis = -1, name = 'concat_layer')([net1, net3])
  dense_1 = layers.Dense(64)
  final_input = dense_1(final_input)
  dense_2 = layers.Dense(64)
  final_input = dense_2(final_input)
  dense_last = layers.Dense(20)
  final_output = dense_last(final_input)

  return Model([text_input1, text_input3], final_output)

In [ ]:
model = make_classifier_model()

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text1 (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 text3 (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 tokenizer1 (KerasLayer)        (None, None, None)   0           ['text1[0][0]']                  
                                                                                                  
 tokenizer3 (KerasLayer)        (None, None, None)   0           ['text3[0][0]']                  
                                                                                              

In [ ]:
model.compile(optimizer = 'adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
final_data_train['sentences'][0]

'days of age a control group cont remained with the mother in the nest during this time cold subjects were developmentally delayed and had lower body and brain weights than the other three groups into adulthood warm and agit subjects both maternally separated at nest temperature had significant growth delays compared to cont but grew more quickly than cold subjects cold subjects were less AS than the other maternally separated groups and warm and agit groups were more active activity did not differ at or days of age however AD warm subjects were less sensitive and cold subjects were more sensitive to amphetamine as measured by locomotor activity than cont and agit subjects who did not differ from each other the relationship between early stress changes in'

In [ ]:
final_data_train['words'][0]

'AS'

In [ ]:
train1 = np.array(final_data_train['sentences'].tolist())
train2 = np.array(final_data_train['words'].tolist())
train3 = np.array(final_data_train['sentences'].tolist())

In [ ]:
final_data_train_labels

array([ 8,  8, 17, ..., 11, 17, 11])

In [ ]:
model.fit(x = [train1, train3], y = final_data_train_labels, epochs = 30)

In [ ]:
# model.fit(x = [train1, train3], y = final_data_train_labels, epochs = 100)

In [ ]:
test1 = np.array(final_data_test['sentences'].tolist())
test2 = np.array(final_data_test['words'].tolist())
test3 = np.array(final_data_test['sentnces'].tolist())

In [ ]:
test_pred = model.predict([test1,test3])

In [ ]:
final_prediction = []

for i in range(0, len(test_pred)):
  if (test_pred[i] > 0.5):
    final_prediction.append(1)
  else:
    final_prediction.append(0)

In [ ]:
import tqdm.notebook as tq
tp=0
fp=0
tn=0
fn=0
for i in tq.tqdm(range(0,len(final_prediction))):
    if (final_prediction[i]==1 and final_labels_test[i]==1):
      tp=tp+1
    elif (final_prediction[i]==0 and final_labels_test[i]==1):
      fp=fp+1
    elif (final_prediction[i]==1 and final_labels_test[i]==0):
      fn=fn+1
    elif (final_prediction[i]==0 and final_labels_test[i]==0):
      tn=tn+1
    
precision_nolower = tp/(tp+fp)
print("Precision = ",precision_nolower)

recall_nolower = tp/(tp+fn)
print("Recall = ",recall_nolower)

f1_nolower = (2*precision_nolower*recall_nolower)/(precision_nolower+recall_nolower)
print("F1 Score = ",f1_nolower)

# Training on SciAI and testing on MeDal(test set)

In [ ]:
data2_train=pd.read_json('https://raw.githubusercontent.com/amirveyseh/AAAI-21-SDU-shared-task-1-AI/master/dataset/train.json')

In [ ]:
d_train1=[] # not abb
d_train1_context=[]
d_train_label1=[]

d_train2=[] # abb
d_train2_context=[]
d_train_label2=[]

In [ ]:
import tqdm.notebook as tq
for i in tq.tqdm(range(0,len(data2_train))):
    for j in range(0,len(data2_train['tokens'][i])):
        if(data2_train['labels'][i][j]!='B-short'):
            d_train1.append(str(data2_train['tokens'][i][j]))
            left = max(0,j-4)
            right = min(len(data2_train['tokens'][i])-1 , j+4)
            sen = (" ").join(data2_train['tokens'][i][left:(right+1)])
            d_train1_context.append(sen)
            d_train_label1.append(0)
        else:
            d_train2.append(str(data2_train['tokens'][i][j]))
            left = max(0,j-4)
            right = min(len(data2_train['tokens'][i])-1 , j+4)
            sen = (" ").join(data2_train['tokens'][i][left:(right+1)])
            d_train2_context.append(sen)
            d_train_label2.append(1)

  0%|          | 0/14006 [00:00<?, ?it/s]

In [ ]:
print(len(d_train1))
print(len(d_train1_context))
print(len(d_train_label1))

print(len(d_train2))
print(len(d_train2_context))
print(len(d_train_label2))

428739
428739
428739
24176
24176
24176


In [ ]:
d_train3=[] # not abb
d_train3_context=[]
d_train_label3=[]

for i in tq.tqdm(range(0,len(d_train1))):
    if(i%3 ==0):
        d_train3.append(d_train1[i])
        d_train3_context.append(d_train1_context[i])
        d_train_label3.append(d_train_label1[i])
for i in tq.tqdm(range(0,len(d_train2))):
    d_train3.append(d_train2[i])
    d_train3_context.append(d_train2_context[i])
    d_train_label3.append(d_train_label2[i])  
    
d_train3 = np.array(d_train3)
d_train3_context = np.array(d_train3_context)
d_train_label3 = np.array(d_train_label3)

for i in tq.tqdm(range(0,len(d_train3))):
    d_train3[i]=str(d_train3[i])
    d_train3_context[i]=str(d_train3_context[i])
    d_train_label3[i]=str(d_train_label3[i])
    
print(d_train3.shape)
print(d_train3_context.shape)
print(d_train_label3.shape)

  0%|          | 0/428739 [00:00<?, ?it/s]

  0%|          | 0/24176 [00:00<?, ?it/s]

  0%|          | 0/167089 [00:00<?, ?it/s]

(167089,)
(167089,)
(167089,)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

preprocess_lnk = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
enc_lnk = 'https://tfhub.dev/google/experts/bert/pubmed/2'
enc_lnk2 = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'

bert_preprocessor = hub.load(preprocess_lnk)

def make_classifier_model2():
  text_input1 = layers.Input(shape = (), dtype = tf.string, name = 'text1')
  tokenizer_bert1 = hub.KerasLayer(bert_preprocessor.tokenize, name = 'tokenizer1')
  segments1 = tokenizer_bert1(text_input1)
  packer1 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs, arguments = dict(seq_length = 30))
  encoder_inputs1 = packer1([segments1])
  bert_encoder1 = hub.KerasLayer(enc_lnk2, trainable = False, name = 'BERT_encoder1')
  out1 = bert_encoder1(encoder_inputs1)
  net1 = out1['sequence_output']
  model1 = tf.keras.Sequential()
  model1.add(layers.LSTM(100,return_sequences=True))
  model1.add(layers.LSTM(40,return_sequences=True))
  model1.add(layers.LSTM(10))
  net1 = model1(net1)

  text_input2 = layers.Input(shape = (), dtype = tf.string, name = 'text2')
  tokenizer_bert2 = hub.KerasLayer(bert_preprocessor.tokenize, name = 'tokenizer2')
  segments2 = tokenizer_bert2(text_input2)
  packer2 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs, arguments = dict(seq_length = 5))
  encoder_inputs2 = packer2([segments2])
  bert_encoder2 = hub.KerasLayer(enc_lnk2, trainable = False, name = 'BERT_encoder2')
  out2 = bert_encoder2(encoder_inputs2)
  net2 = out2['sequence_output']
  model2 = tf.keras.Sequential()
  model2.add(layers.LSTM(100,return_sequences=True))
  model2.add(layers.LSTM(40,return_sequences=True))
  model2.add(layers.LSTM(10))
  net2 = model2(net2)

  text_input3 = layers.Input(shape = (), dtype = tf.string, name = 'text3')
  tokenizer_bert3 = hub.KerasLayer(bert_preprocessor.tokenize, name = 'tokenizer3')
  segments3 = tokenizer_bert3(text_input3)
  packer3 = hub.KerasLayer(bert_preprocessor.bert_pack_inputs, arguments = dict(seq_length = 5))
  encoder_inputs3 = packer3([segments3])
  bert_encoder3 = hub.KerasLayer(enc_lnk, trainable = False, name = 'BERT_encoder3')
  out3 = bert_encoder3(encoder_inputs3)
  net3 = out3['sequence_output']
  model3 = tf.keras.Sequential()
  model3.add(layers.LSTM(100,return_sequences=True))
  model3.add(layers.LSTM(40,return_sequences=True))
  model3.add(layers.LSTM(10))
  net3 = model3(net3)

  final_input = layers.Concatenate(axis = -1, name = 'concat_layer')([net1, net2, net3])
  dense_last = layers.Dense(1, activation = 'sigmoid')
  final_output = dense_last(final_input)

  return Model([text_input1, text_input2, text_input3], final_output)

In [ ]:
model2 = make_classifier_model2()

In [ ]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text1 (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
text2 (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
text3 (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
tokenizer1 (KerasLayer)         (None, None, None)   0           text1[0][0]                      
____________________________________________________________________________________________

In [ ]:
type(d_train3)

numpy.ndarray

In [ ]:
model2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
d_train3_cpy = d_train3.copy()

In [ ]:
print(len(d_train3))
print(len(d_train3_cpy))

167089
167089


In [ ]:
model2.fit(x = [d_train3_context,d_train3,d_train3_cpy], y = d_train_label3 , epochs = 5)

Epoch 1/5
5222/5222 [==============================] - 678s 127ms/step - loss: 0.0760 - precision_1: 0.9057 - recall_1: 0.9173
Epoch 2/5
5222/5222 [==============================] - 661s 127ms/step - loss: 0.0581 - precision_1: 0.9243 - recall_1: 0.9440
Epoch 3/5
5222/5222 [==============================] - 661s 127ms/step - loss: 0.0531 - precision_1: 0.9288 - recall_1: 0.9488
Epoch 4/5
5222/5222 [==============================] - 660s 126ms/step - loss: 0.0498 - precision_1: 0.9328 - recall_1: 0.9538
Epoch 5/5
5222/5222 [==============================] - 660s 126ms/step - loss: 0.0476 - precision_1: 0.9355 - recall_1: 0.9571


In [ ]:
# test_pred2 = model.predict([test1,test2,test3])

In [ ]:
# final_prediction2 = []

# for i in range(0, len(test_pred2)):
#   if (test_pred2[i] > 0.5):
#     final_prediction2.append(1)
#   else:
#     final_prediction2.append(0)

In [ ]:
# import tqdm.notebook as tq
# tp=0
# fp=0
# tn=0
# fn=0
# for i in tq.tqdm(range(0,len(final_prediction2))):
#     if (final_prediction2[i]==1 and final_labels_test[i]==1):
#       tp=tp+1
#     elif (final_prediction2[i]==0 and final_labels_test[i]==1):
#       fp=fp+1
#     elif (final_prediction2[i]==1 and final_labels_test[i]==0):
#       fn=fn+1
#     elif (final_prediction2[i]==0 and final_labels_test[i]==0):
#       tn=tn+1
    
# precision_nolower = tp/(tp+fp)
# print("Precision = ",precision_nolower)

# recall_nolower = tp/(tp+fn)
# print("Recall = ",recall_nolower)

# f1_nolower = (2*precision_nolower*recall_nolower)/(precision_nolower+recall_nolower)
# print("F1 Score = ",f1_nolower)

  0%|          | 0/177242 [00:00<?, ?it/s]

Precision =  0.9041533546325878
Recall =  0.8141542002301496
F1 Score =  0.8567968513472601


In [ ]:
final_data.to_csv('final_data_abb_notebook2.csv', index = False)

In [ ]:
test_pred2 = model2.predict([test1,test2,test3])

In [ ]:
final_prediction2 = []

for i in range(0, len(test_pred2)):
  if (test_pred2[i] > 0.5):
    final_prediction2.append(1)
  else:
    final_prediction2.append(0)

In [ ]:
import tqdm.notebook as tq
tp=0
fp=0
tn=0
fn=0
for i in tq.tqdm(range(0,len(final_prediction2))):
    if (final_prediction2[i]==1 and final_labels_test[i]==1):
      tp=tp+1
    elif (final_prediction2[i]==0 and final_labels_test[i]==1):
      fp=fp+1
    elif (final_prediction2[i]==1 and final_labels_test[i]==0):
      fn=fn+1
    elif (final_prediction2[i]==0 and final_labels_test[i]==0):
      tn=tn+1
    
precision_nolower = tp/(tp+fp)
print("Precision = ",precision_nolower)

recall_nolower = tp/(tp+fn)
print("Recall = ",recall_nolower)

f1_nolower = (2*precision_nolower*recall_nolower)/(precision_nolower+recall_nolower)
print("F1 Score = ",f1_nolower)

  0%|          | 0/177242 [00:00<?, ?it/s]

Precision =  0.899520766773163
Recall =  0.48140548858681714
F1 Score =  0.6271648939132372
